In [1]:
#importing libraries
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import math

In [2]:
#load the NSL_KDD dataset
train=pd.read_csv('dataset/Train_ICMP1.csv')
test=pd.read_csv('dataset/Test_ICMP1.csv')

#size of train and test data
print(train.shape,test.shape)

# concatenate traina and test data
dataset=pd.concat([test,train])

#size of whole dataset
print(dataset.shape)

#print(dataset.head(2))
#print(dataset.columns)
X = dataset.iloc[:, :-1].values
print(X)
y = dataset.iloc[:, 41:42].values
print(y)

(4156, 42) (15451, 42)
(19607, 42)
[[0 'tcp' 'private' ... 0.0 1.0 1.0]
 [0 'tcp' 'private' ... 0.0 1.0 1.0]
 [2 'tcp' 'ftp_data' ... 0.0 0.0 0.0]
 ...
 [0 'icmp' 'eco_i' ... 0.0 0.0 0.0]
 [0 'icmp' 'ecr_i' ... 0.0 0.0 0.0]
 [0 'icmp' 'eco_i' ... 0.0 0.0 0.0]]
[['DDoS']
 ['DDoS']
 ['normal']
 ...
 ['normal']
 ['normal']
 ['normal']]


In [3]:
#Encoding of dataset
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1, 2, 3])], remainder = 'passthrough')
X= ct.fit_transform(X)
print(X)

[[0.0 1.0 0.0 ... 0.0 1.0 1.0]
 [0.0 1.0 0.0 ... 0.0 1.0 1.0]
 [0.0 1.0 0.0 ... 0.0 0.0 0.0]
 ...
 [1.0 0.0 0.0 ... 0.0 0.0 0.0]
 [1.0 0.0 0.0 ... 0.0 0.0 0.0]
 [1.0 0.0 0.0 ... 0.0 0.0 0.0]]


In [4]:
from sklearn.preprocessing import LabelEncoder
le_y = LabelEncoder()
y[:, 0] = le_y.fit_transform(y[:, 0])
y= y.astype(int)
y = y.ravel()
print(y)

[0 0 1 ... 1 1 1]


In [5]:
# min_max normalisation between (a,b)
from sklearn.preprocessing import minmax_scale
X = minmax_scale(X, feature_range=(0,1), axis=0)
print(X)

[[0. 1. 0. ... 0. 1. 1.]
 [0. 1. 0. ... 0. 1. 1.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [6]:
# function to calculate shannon entropy
def cal_entropy(column):
   counts=np.bincount(column)
   probabilites=counts / len(column)
   entropy=0
   for prob in probabilites:
      if(prob)>0:
        entropy+=prob*math.log(prob,2)
   return -entropy 

In [7]:
# function to calculte MAD adaptive threshold technique
def thres_MAD(data):
  a=3
  from numpy import mean, absolute
  MAD =mean(absolute(data - mean(data)))
  thres= mean(data)+a*MAD
  return thres

In [8]:
print(X.shape,y.shape)

(19607, 114) (19607,)


In [9]:
# form a list of attribute that has to deleted further
list=[]
for j in range(0,114):
   col=np.array(X[:,j])
   col=col.astype(int)
   if  cal_entropy(col)<thres_MAD(col):
           list.append(j)


print(list)

[0, 1, 2, 14, 16, 17, 25, 47, 52, 62, 67, 71, 75, 84, 95, 96, 97, 98, 99, 100, 101, 103, 104, 105, 106, 108, 110, 111, 112, 113]


In [10]:
# delete the attribute that have less entropy than adaptive threshold value(THR)
X = np.delete(X, list, axis=1)

In [11]:
print(X.shape)

(19607, 84)


In [12]:
# splitting of dataset in train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)
print(X_train)
print(X_test)
print(y_train)
print(y_test)


[[0.   0.   0.   ... 0.   0.03 0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.39 0.73 0.  ]
 ...
 [0.   0.   0.   ... 0.   0.   0.06]
 [0.   0.   0.   ... 0.02 0.01 0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]]
[[0.   0.   0.   ... 0.05 0.05 0.  ]
 [0.   0.   0.   ... 0.   0.   0.04]
 [0.   0.   0.   ... 0.1  0.51 0.  ]
 ...
 [0.   0.   0.   ... 0.   0.06 0.  ]
 [0.   0.   0.   ... 0.07 0.07 0.  ]
 [0.   0.   0.   ... 0.   0.   0.04]]
[0 1 1 ... 1 1 0]
[0 1 0 ... 0 0 1]


In [13]:
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

In [14]:
# base classifier RandomForest
y_train = y_train.ravel()
clf1 = RandomForestClassifier(n_estimators=100)
clf1.fit(X_train, y_train)
y_pred1 = clf1.predict(X_test)

from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(y_test, y_pred1)
cm.print_stats()
#Root mean square error between actual and predicted class
from sklearn.metrics import mean_squared_error
from math import sqrt
RMSE = sqrt(mean_squared_error(y_test, y_pred1))
print("RMSE= ",RMSE)

population: 5883
P: 3324
N: 2559
PositiveTest: 3321
NegativeTest: 2562
TP: 3315
TN: 2553
FP: 6
FN: 9
TPR: 0.9972924187725631
TNR: 0.9976553341148886
PPV: 0.998193315266486
NPV: 0.9964871194379391
FPR: 0.0023446658851113715
FDR: 0.0018066847335140017
FNR: 0.002707581227436823
ACC: 0.9974502804691484
F1_score: 0.9977426636568849
MCC: 0.9948140848169982
informedness: 0.9949477528874517
markedness: 0.9946804347044251
prevalence: 0.565017848036716
LRP: 425.3452166064982
LRN: 0.0027139445205682845
DOR: 156725.83333333334
FOR: 0.00351288056206089
RMSE=  0.05049474755706386


In [15]:
# base classifier GradientBoosting
clf2 = GradientBoostingClassifier(n_estimators=100)
clf2.fit(X_train, y_train)
y_pred2 = clf2.predict(X_test)

from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(y_test, y_pred2)
cm.print_stats()
#Root mean square error between actual and predicted class
from sklearn.metrics import mean_squared_error
from math import sqrt
RMSE = sqrt(mean_squared_error(y_test, y_pred2))
print("RMSE= ",RMSE)

population: 5883
P: 3324
N: 2559
PositiveTest: 3320
NegativeTest: 2563
TP: 3313
TN: 2552
FP: 7
FN: 11
TPR: 0.996690734055355
TNR: 0.9972645564673701
PPV: 0.9978915662650603
NPV: 0.9957081545064378
FPR: 0.0027354435326299334
FDR: 0.002108433734939759
FNR: 0.003309265944645006
ACC: 0.9969403365629781
F1_score: 0.9972907886815171
MCC: 0.993777489744426
informedness: 0.9939552905227251
markedness: 0.9935997207714982
prevalence: 0.565017848036716
LRP: 364.36165549252195
LRN: 0.0033183430847753014
DOR: 109802.28571428574
FOR: 0.004291845493562232
RMSE=  0.05531422454506551


In [16]:
# SVM
from sklearn.svm import SVC  
clf3= SVC(kernel='linear') 
clf3.fit(X_train, y_train) 
y_pred3 = clf3.predict(X_test)

from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(y_test, y_pred3)
cm.print_stats()
#Root mean square error between actual and predicted class
from sklearn.metrics import mean_squared_error
from math import sqrt
RMSE = sqrt(mean_squared_error(y_test, y_pred3))
print("RMSE= ",RMSE)

population: 5883
P: 3324
N: 2559
PositiveTest: 4897
NegativeTest: 986
TP: 3165
TN: 827
FP: 1732
FN: 159
TPR: 0.9521660649819494
TNR: 0.32317311449785074
PPV: 0.6463140698386768
NPV: 0.8387423935091278
FPR: 0.6768268855021493
FDR: 0.35368593016132327
FNR: 0.04783393501805054
ACC: 0.6785653578106409
F1_score: 0.7699793212504561
MCC: 0.3654518417241296
informedness: 0.27533917947980013
markedness: 0.4850564633478047
prevalence: 0.565017848036716
LRP: 1.4068088685270257
LRN: 0.14801334910664005
DOR: 9.504608043923483
FOR: 0.1612576064908722
RMSE=  0.5669520633963326


In [17]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
clf4 = KNeighborsClassifier(n_neighbors = 17)
clf4.fit(X_train, y_train) 
y_pred4 = clf4.predict(X_test)

from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(y_test, y_pred4)
cm.print_stats()
#Root mean square error between actual and predicted class
from sklearn.metrics import mean_squared_error
from math import sqrt
RMSE = sqrt(mean_squared_error(y_test, y_pred4))
print("RMSE= ",RMSE)

population: 5883
P: 3324
N: 2559
PositiveTest: 3279
NegativeTest: 2604
TP: 3250
TN: 2530
FP: 29
FN: 74
TPR: 0.9777376654632972
TNR: 0.9886674482219617
PPV: 0.9911558401951814
NPV: 0.9715821812596006
FPR: 0.011332551778038297
FDR: 0.008844159804818542
FNR: 0.022262334536702767
ACC: 0.9824919258881523
F1_score: 0.9844010298349235
MCC: 0.9645698248821232
informedness: 0.9664051136852589
markedness: 0.962738021454782
prevalence: 0.565017848036716
LRP: 86.27692020415785
LRN: 0.022517515446412006
DOR: 3831.547064305685
FOR: 0.028417818740399385
RMSE=  0.13231807930833828


In [18]:
# meta classifier LogisticRegression
lr = LogisticRegression(solver='lbfgs')


In [19]:
# stack esemble method 
from mlxtend.classifier import StackingClassifier
sclf = StackingClassifier(classifiers=[clf1, clf2,clf4], 
                          meta_classifier=lr)

In [20]:
# train and test the model
sclf.fit(X_train, y_train)
y_pred = sclf.predict(X_test)

from pandas_ml import ConfusionMatrix
cm = ConfusionMatrix(y_test, y_pred)
cm.print_stats()
#Root mean square error between actual and predicted class
from sklearn.metrics import mean_squared_error
from math import sqrt
RMSE = sqrt(mean_squared_error(y_test, y_pred))
print("RMSE= ",RMSE)


population: 5883
P: 3324
N: 2559
PositiveTest: 3322
NegativeTest: 2561
TP: 3316
TN: 2553
FP: 6
FN: 8
TPR: 0.9975932611311673
TNR: 0.9976553341148886
PPV: 0.9981938591210114
NPV: 0.996876220226474
FPR: 0.0023446658851113715
FDR: 0.001806140878988561
FNR: 0.0024067388688327317
ACC: 0.9976202617712052
F1_score: 0.9978934697562444
MCC: 0.9951593332939035
informedness: 0.9952485952460559
markedness: 0.9950700793474856
prevalence: 0.565017848036716
LRP: 425.4735258724429
LRN: 0.002412395129394031
DOR: 176369.75
FOR: 0.0031237797735259665
RMSE=  0.04878256070354274
